In [1]:
import os 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import h5py
import numpy as np
import glob
import shutil
matplotlib.rcParams['figure.figsize'] = [15, 15]
pd.options.mode.chained_assignment = None

In [15]:
def save_hdf5(output_path, asset_dict, attr_dict=None, mode="a"):
    file = h5py.File(output_path, mode)
    for key, val in asset_dict.items():
        data_shape = val.shape
        if key not in file:
            data_type = val.dtype
            chunk_shape = (1,) + data_shape[1:]
            maxshape = (None,) + data_shape[1:]
            dset = file.create_dataset(
                key,
                shape=data_shape,
                maxshape=maxshape,
                chunks=chunk_shape,
                dtype=data_type,
            )
            dset[:] = val
            if attr_dict is not None:
                if key in attr_dict.keys():
                    for attr_key, attr_val in attr_dict[key].items():
                        dset.attrs[attr_key] = attr_val
        else:
            dset = file[key]
            dset.resize(len(dset) + data_shape[0], axis=0)
            dset[-data_shape[0] :] = val
    file.close()
    return output_path

In [2]:
bt_proj = pd.read_csv("/home/mathiane/LNENWork/LNEN_LCNEC_Molecular_barlow_twin/projector/concat_proj_by_patient/barlowTwins_largeproj_LNEN_LCNEC_all_CLEAN.csv")

/home/mathiane/miniconda3/envs/HaloAE/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (131,132,137,138,139,140) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
bt_proj.head()

,Unnamed: 0,Unnamed: 0_x,0,1,2,3,4,5,6,7,...,y,img_id_c,Unnamed: 0_y,manuscript_id,centre_patient_id,TNEID,tiles_folder_y,archtype_label_combined,case_number,main_vs_dupliated_slides
0,0,0,0.338309,-0.291670,0.802182,-0.451597,0.378702,0.034748,-0.619565,0.420880,...,15745,TNE0892_77953_15745,268,LNEN179,Lca-827,TNE0892,TNE0892,Ca A1,223.0,main
1,1,1,-0.116351,-0.687656,0.527643,0.061367,0.845914,-0.355424,-0.981373,0.071839,...,25345,TNE0892_71809_25345,268,LNEN179,Lca-827,TNE0892,TNE0892,Ca A1,223.0,main
2,2,2,0.352333,0.106918,-0.714990,0.357960,-0.666445,-0.351016,-1.053463,0.726573,...,48385,TNE0892_76033_48385,268,LNEN179,Lca-827,TNE0892,TNE0892,Ca A1,223.0,main
3,3,3,-0.505866,-0.598584,-0.424595,0.751148,0.160364,-0.259544,-0.287292,-0.144396,...,26881,TNE0892_84097_26881,268,LNEN179,Lca-827,TNE0892,TNE0892,Ca A1,223.0,main
4,4,4,-0.290795,0.148661,0.088907,-0.234824,-0.456991,1.009766,0.060430,-0.527327,...,6145,TNE0892_57601_6145,268,LNEN179,Lca-827,TNE0892,TNE0892,Ca A1,223.0,main


In [4]:
bt_proj = bt_proj.drop(columns=["Unnamed: 0" ,"Unnamed: 0_x",	"Unnamed: 0_y"])

# 1- CaA1  vs CaA2

In [9]:
bt_proj_select = bt_proj[(bt_proj['main_vs_dupliated_slides']=='main') & (bt_proj['archtype_label_combined'].isin(["Ca A1", "Ca A2"]))]

In [12]:
bt_proj_select.groupby("sample_id").size()

sample_id
TNE0001     7170
TNE0003     5915
TNE0007    10706
TNE0008     7944
TNE0009    45150
           ...  
TNE1458     5491
TNE1499    29410
TNE1508    23762
TNE1794    25238
TNE1903     5201
Length: 135, dtype: int64

In [13]:
bt_proj_select.groupby("archtype_label_combined").size()

archtype_label_combined
Ca A1     984402
Ca A2    1006260
dtype: int64

In [14]:
bt_proj_select[["sample_id","archtype_label_combined"]].drop_duplicates().groupby("archtype_label_combined").size()

archtype_label_combined
Ca A1    74
Ca A2    61
dtype: int64

In [16]:
bt_proj_select.head()

,0,1,2,3,4,5,6,7,8,9,...,x,y,img_id_c,manuscript_id,centre_patient_id,TNEID,tiles_folder_y,archtype_label_combined,case_number,main_vs_dupliated_slides
0,0.338309,-0.291670,0.802182,-0.451597,0.378702,0.034748,-0.619565,0.420880,0.054787,0.264495,...,77953,15745,TNE0892_77953_15745,LNEN179,Lca-827,TNE0892,TNE0892,Ca A1,223.0,main
1,-0.116351,-0.687656,0.527643,0.061367,0.845914,-0.355424,-0.981373,0.071839,-0.058305,0.118359,...,71809,25345,TNE0892_71809_25345,LNEN179,Lca-827,TNE0892,TNE0892,Ca A1,223.0,main
2,0.352333,0.106918,-0.714990,0.357960,-0.666445,-0.351016,-1.053463,0.726573,-0.020366,0.025735,...,76033,48385,TNE0892_76033_48385,LNEN179,Lca-827,TNE0892,TNE0892,Ca A1,223.0,main
3,-0.505866,-0.598584,-0.424595,0.751148,0.160364,-0.259544,-0.287292,-0.144396,0.492094,0.141897,...,84097,26881,TNE0892_84097_26881,LNEN179,Lca-827,TNE0892,TNE0892,Ca A1,223.0,main
4,-0.290795,0.148661,0.088907,-0.234824,-0.456991,1.009766,0.060430,-0.527327,0.831172,0.120493,...,57601,6145,TNE0892_57601_6145,LNEN179,Lca-827,TNE0892,TNE0892,Ca A1,223.0,main


In [19]:
bt_proj_select.iloc[:,128:]

,img_id,sample_id,tiles_folder_x,x,y,img_id_c,manuscript_id,centre_patient_id,TNEID,tiles_folder_y,archtype_label_combined,case_number,main_vs_dupliated_slides
0,TNE0892_77953_15745,TNE0892,TNE0892,77953,15745,TNE0892_77953_15745,LNEN179,Lca-827,TNE0892,TNE0892,Ca A1,223.0,main
1,TNE0892_71809_25345,TNE0892,TNE0892,71809,25345,TNE0892_71809_25345,LNEN179,Lca-827,TNE0892,TNE0892,Ca A1,223.0,main
2,TNE0892_76033_48385,TNE0892,TNE0892,76033,48385,TNE0892_76033_48385,LNEN179,Lca-827,TNE0892,TNE0892,Ca A1,223.0,main
3,TNE0892_84097_26881,TNE0892,TNE0892,84097,26881,TNE0892_84097_26881,LNEN179,Lca-827,TNE0892,TNE0892,Ca A1,223.0,main
4,TNE0892_57601_6145,TNE0892,TNE0892,57601,6145,TNE0892_57601_6145,LNEN179,Lca-827,TNE0892,TNE0892,Ca A1,223.0,main
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6419433,TNE1416_47233_44161,TNE1416,TNE1416,47233,44161,TNE1416_47233_44161,LNEN103,LB11-0147,TNE1416,TNE1416,Ca A1,58.0,main
6419434,TNE1416_39937_16513,TNE1416,TNE1416,39937,16513,TNE1416_39937_16513,LNEN103,LB11-0147,TNE1416,TNE1416,Ca A1,58.0,main
6419435,TNE1416_62977_56833,TNE1416,TNE1416,62977,56833,TNE1416_62977_56833,LNEN103,LB11-0147,TNE1416,TNE1416,Ca A1,58.0,main
6419436,TNE1416_28801_53761,TNE1416,TNE1416,28801,53761,TNE1416_28801_53761,LNEN103,LB11-0147,TNE1416,TNE1416,Ca A1,58.0,main


In [23]:
root_data = "/home/mathiane/LNENWork/data_RoFormerMIL_LNEN_LCNEC_training_CaA1CaA2_norm"
os.makedirs(root_data, exist_ok=True)
os.makedirs(os.path.join(root_data, "features_h5_files"), exist_ok=True)
os.makedirs(os.path.join(root_data, "features_h5_files", "CaA1"), exist_ok=True)
os.makedirs(os.path.join(root_data, "features_h5_files", "CaA2"), exist_ok=True)
os.makedirs( os.path.join(root_data, "coords_table"), exist_ok=True)
os.makedirs( os.path.join(root_data, "coords_table", "CaA1"), exist_ok=True)
os.makedirs( os.path.join(root_data, "coords_table", "CaA2"), exist_ok=True)

for tneid in set(bt_proj_select["sample_id"]):
    print("Tneid = ", tneid)
    BTProjArchCaA1CaA2_c = bt_proj_select[bt_proj_select["sample_id"] == tneid]
    BTProjArchCaA1CaA2_c = BTProjArchCaA1CaA2_c.sort_values(by=["x","y"])
    BTProjArchCaA1CaA2_c["x_rank"] = BTProjArchCaA1CaA2_c["x"].rank(method="min")-1
    BTProjArchCaA1CaA2_c["y_rank"] = BTProjArchCaA1CaA2_c["y"].rank(method="min")-1
    BTProjArchCaA1CaA2_c["x_rank"] = BTProjArchCaA1CaA2_c["x_rank"].astype(int)
    BTProjArchCaA1CaA2_c["y_rank"] = BTProjArchCaA1CaA2_c["y_rank"].astype(int)
    
    BTProjArchCaA1CaA2_features =  BTProjArchCaA1CaA2_c.iloc[:,0:128]
    BTProjArchCaA1CaA2_features=(BTProjArchCaA1CaA2_features-BTProjArchCaA1CaA2_features.mean(axis=0))/BTProjArchCaA1CaA2_features.std(axis=0)
    # print("BT normalized_df " , normalized_df.shape, normalized_df.head)
    # print("BT BTProjArchCaA1CaA2_features std " , BTProjArchCaA1CaA2_features.mean(), BTProjArchCaA1CaA2_features.std())
    # mean_over_col = BTProjArchCaA1CaA2_features.mean()
    # std_over_col=  BTProjArchCaA1CaA2_features.std()
    BTProjArchCaA1CaA2_coords = BTProjArchCaA1CaA2_c[["img_id_c", "x","y", "x_rank", "y_rank"]]
    BTProjArchCaA1CaA2_coords_for_export = BTProjArchCaA1CaA2_coords[["x_rank", "y_rank"]]
    #print(BTProjArchCaA1CaA2_coords.head())
    
    archetype_c = list(BTProjArchCaA1CaA2_c["archtype_label_combined"])[0]
    archetype_c = archetype_c.replace(" ", "")
    print(archetype_c)
 
    asset_dict_tumor = {   
        "coords": BTProjArchCaA1CaA2_coords_for_export.to_numpy(),
        "features": BTProjArchCaA1CaA2_features.to_numpy(dtype='float32'),
            }
    
    
    h5_file_tumor = os.path.join(root_data, "features_h5_files", archetype_c, tneid + ".h5" )
    save_hdf5(h5_file_tumor, asset_dict_tumor, attr_dict=None, mode = "w")
    
    coords_tumor_filename = os.path.join(root_data, "coords_table", archetype_c, tneid + "coords.csv" )
    BTProjArchCaA1CaA2_coords.to_csv(coords_tumor_filename, index=False )
    
    print("___________________________________________________ \n")

Tneid =  TNE1007
CaA2
___________________________________________________ 

Tneid =  TNE1431
CaA1
___________________________________________________ 

Tneid =  TNE0963
CaA2
___________________________________________________ 

Tneid =  TNE0408
CaA1
___________________________________________________ 

Tneid =  TNE0286
CaA1
___________________________________________________ 

Tneid =  TNE1027
CaA2
___________________________________________________ 

Tneid =  TNE1089
CaA1
___________________________________________________ 

Tneid =  TNE1019
CaA2
___________________________________________________ 

Tneid =  TNE1008
CaA2
___________________________________________________ 

Tneid =  TNE0993
CaA1
___________________________________________________ 

Tneid =  TNE0979
CaA2
___________________________________________________ 

Tneid =  TNE0954
CaA2
___________________________________________________ 

Tneid =  TNE0803
CaA1
___________________________________________________ 

Tneid =  TNE

In [24]:
# Create slides_format slides
case_id = []
slide_id = []
label_id = []
label = []
for ele in os.listdir(os.path.join(root_data, "features_h5_files", "CaA1")):
    if ele.find("h5") != -1:
        case_id.append(ele.split(".")[0])
        slide_id.append(ele.split(".")[0])
        label_id.append("CaA1")
        label.append("CaA1")
for ele in os.listdir(os.path.join(root_data, "features_h5_files", "CaA2")):
    if ele.find("h5") != -1:
        case_id.append(ele.split(".")[0])
        slide_id.append(ele.split(".")[0])
        label_id.append("CaA2")
        label.append("CaA2")
slides_format = pd.DataFrame({"case_id":case_id,
                              "slide_id": slide_id,
                              "label_id": label_id,
                              "label": label})
print(slides_format.head())
os.makedirs(os.path.join(root_data, "dataRoFormer"), exist_ok=True)

slides_format.to_csv( os.path.join(root_data, "dataRoFormer", "slides_format.csv"))


   case_id slide_id label_id label
0  TNE1794  TNE1794     CaA1  CaA1
1  TNE0993  TNE0993     CaA1  CaA1
2  TNE1448  TNE1448     CaA1  CaA1
3  TNE0889  TNE0889     CaA1  CaA1
4  TNE0001  TNE0001     CaA1  CaA1


In [25]:
root_data

'/home/mathiane/LNENWork/data_RoFormerMIL_LNEN_LCNEC_training_CaA1CaA2_norm'

# 2- CaA1  vs CaA2 vs CaB

In [26]:
bt_proj_select = bt_proj[(bt_proj['main_vs_dupliated_slides']=='main') & (bt_proj['archtype_label_combined'].isin(["Ca A1", "Ca A2", "Ca B"]))]
bt_proj_select[["sample_id","archtype_label_combined"]].drop_duplicates().groupby("archtype_label_combined").size()

archtype_label_combined
Ca A1    74
Ca A2    61
Ca B     50
dtype: int64

In [28]:
root_data = "/home/mathiane/LNENWork/data_RoFormerMIL_LNEN_LCNEC_training_CaA1CaA2CaB_norm"
os.makedirs(root_data, exist_ok=True)
os.makedirs(os.path.join(root_data, "features_h5_files"), exist_ok=True)
os.makedirs(os.path.join(root_data, "features_h5_files", "CaA1"), exist_ok=True)
os.makedirs(os.path.join(root_data, "features_h5_files", "CaA2"), exist_ok=True)
os.makedirs(os.path.join(root_data, "features_h5_files", "CaB"), exist_ok=True)

os.makedirs( os.path.join(root_data, "coords_table"), exist_ok=True)
os.makedirs( os.path.join(root_data, "coords_table", "CaA1"), exist_ok=True)
os.makedirs( os.path.join(root_data, "coords_table", "CaA2"), exist_ok=True)
os.makedirs(os.path.join(root_data, "coords_table", "CaB"), exist_ok=True)


for tneid in set(bt_proj_select["sample_id"]):
    print("Tneid = ", tneid)
    BTProjArchCaA1CaA2_c = bt_proj_select[bt_proj_select["sample_id"] == tneid]
    BTProjArchCaA1CaA2_c = BTProjArchCaA1CaA2_c.sort_values(by=["x","y"])
    BTProjArchCaA1CaA2_c["x_rank"] = BTProjArchCaA1CaA2_c["x"].rank(method="min")-1
    BTProjArchCaA1CaA2_c["y_rank"] = BTProjArchCaA1CaA2_c["y"].rank(method="min")-1
    BTProjArchCaA1CaA2_c["x_rank"] = BTProjArchCaA1CaA2_c["x_rank"].astype(int)
    BTProjArchCaA1CaA2_c["y_rank"] = BTProjArchCaA1CaA2_c["y_rank"].astype(int)
    
    BTProjArchCaA1CaA2_features =  BTProjArchCaA1CaA2_c.iloc[:,0:128]
    BTProjArchCaA1CaA2_features=(BTProjArchCaA1CaA2_features-BTProjArchCaA1CaA2_features.mean(axis=0))/BTProjArchCaA1CaA2_features.std(axis=0)
    # print("BT normalized_df " , normalized_df.shape, normalized_df.head)
    # print("BT BTProjArchCaA1CaA2_features std " , BTProjArchCaA1CaA2_features.mean(), BTProjArchCaA1CaA2_features.std())
    # mean_over_col = BTProjArchCaA1CaA2_features.mean()
    # std_over_col=  BTProjArchCaA1CaA2_features.std()
    BTProjArchCaA1CaA2_coords = BTProjArchCaA1CaA2_c[["img_id_c", "x","y", "x_rank", "y_rank"]]
    BTProjArchCaA1CaA2_coords_for_export = BTProjArchCaA1CaA2_coords[["x_rank", "y_rank"]]
    #print(BTProjArchCaA1CaA2_coords.head())
    
    archetype_c = list(BTProjArchCaA1CaA2_c["archtype_label_combined"])[0]
    archetype_c = archetype_c.replace(" ", "")
    print(archetype_c)
 
    asset_dict_tumor = {   
        "coords": BTProjArchCaA1CaA2_coords_for_export.to_numpy(),
        "features": BTProjArchCaA1CaA2_features.to_numpy(dtype='float32'),
            }
    
    
    h5_file_tumor = os.path.join(root_data, "features_h5_files", archetype_c, tneid + ".h5" )
    save_hdf5(h5_file_tumor, asset_dict_tumor, attr_dict=None, mode = "w")
    
    coords_tumor_filename = os.path.join(root_data, "coords_table", archetype_c, tneid + "coords.csv" )
    BTProjArchCaA1CaA2_coords.to_csv(coords_tumor_filename, index=False )
    
    print("___________________________________________________ \n")

Tneid =  TNE1007
CaA2
___________________________________________________ 

Tneid =  TNE0808
CaB
___________________________________________________ 

Tneid =  TNE1431
CaA1
___________________________________________________ 

Tneid =  TNE0234
CaB
___________________________________________________ 

Tneid =  TNE0963
CaA2
___________________________________________________ 

Tneid =  TNE0408
CaA1
___________________________________________________ 

Tneid =  TNE0286
CaA1
___________________________________________________ 

Tneid =  TNE0342
CaB
___________________________________________________ 

Tneid =  TNE1027
CaA2
___________________________________________________ 

Tneid =  TNE0961
CaB
___________________________________________________ 

Tneid =  TNE1089
CaA1
___________________________________________________ 

Tneid =  TNE1433
CaB
___________________________________________________ 

Tneid =  TNE1436
CaB
___________________________________________________ 

Tneid =  TNE1019
C

In [ ]:
# Create slides_format slides
case_id = []
slide_id = []
label_id = []
label = []
for ele in os.listdir(os.path.join(root_data, "features_h5_files", "CaA1")):
    if ele.find("h5") != -1:
        case_id.append(ele.split(".")[0])
        slide_id.append(ele.split(".")[0])
        label_id.append("CaA1")
        label.append("CaA1")
for ele in os.listdir(os.path.join(root_data, "features_h5_files", "CaA2")):
    if ele.find("h5") != -1:
        case_id.append(ele.split(".")[0])
        slide_id.append(ele.split(".")[0])
        label_id.append("CaA2")
        label.append("CaA2")
slides_format = pd.DataFrame({"case_id":case_id,
                              "slide_id": slide_id,
                              "label_id": label_id,
                              "label": label})
print(slides_format.head())
os.makedirs(os.path.join(root_data, "dataRoFormer"), exist_ok=True)

slides_format.to_csv( os.path.join(root_data, "dataRoFormer", "slides_format.csv"))
